In [28]:
import pandas as pd
import tensorflow as tf
import numpy as np 
from sklearn.model_selection import train_test_split

In [49]:
pitches = pd.read_csv("pitchesFinalDF.csv")

/Users/JeffyLands/182/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
input_cols = ['PAOFINNING', 'PITCHOFPA', 'Pitcher Id Norm', 'PITCHERTHROWS', 'BATTERSIDE', 'INNING', 'OUTS', 
             'BALLS', 'STRIKES', 'PitchOutcome', 'RELSPEED', 'VERTRELANGLE', 'HORZRELANGLE', 'SPINRATE',
             'SPINAXIS', 'RELHEIGHT', 'RELSIDE', 'EXTENSION', 'VERTBREAK', 'INDUCEDVERTBREAK', 'HORZBREAK',
             'PLATELOCHEIGHT', 'PLATELOCSIDE', 'VERTAPPRANGLE', 'HORZAPPRANGLE', 'PFXX', 'PFXZ', 'X0', 'Z0',
             'VX0', 'VY0', 'VZ0', 'AX0', 'AY0', 'AZ0', 'Batter Id Norm', 'NEXT BALLS', 'NEXT STRIKES', 'NEXT OUTS',
             'NEXT INNING', 'PA NEXT', 'NEXT PITCHERID', 'Pitch Type', 'Next Pitch Label']

In [58]:
pitches['values'] = pitches[input_cols[:len(input_cols) - 1]].apply(lambda x: list(x), axis=1)

In [71]:
pitches = pitches[pitches['Next Pitch Label'].notnull()]

In [153]:
pitches['label'] = pitches['Next Pitch Label'] + 1

In [155]:
pitches.groupby(["PI_PITCH_TYPE", "Pitch Type"]).count()

,,Unnamed: 0,PITCHNO,TM_DATE_TIME,PAOFINNING,PITCHOFPA,PITCHER,PITCHERID,PITCHERTHROWS,BATTER,BATTERID,...,NEXT STRIKES,NEXT BATTER,NEXT BATTERID,NEXT OUTS,NEXT INNING,PA NEXT,NEXT PITCHERID,Next Pitch Label,values,label
PI_PITCH_TYPE,Pitch Type,,,,,,,,,,,,,,,,,,,,,
CH,0,75489,75489,75488,75489,75489,75489,75489,75489,75489,75489,...,75489,75489,75489,75489,75489,75489,75489,75489,75489,75489
CS,1,198,198,198,198,198,198,198,198,198,198,...,198,198,198,198,198,198,198,198,198,198
CU,2,73634,73634,73633,73634,73634,73634,73634,73634,73634,73634,...,73634,73634,73634,73634,73634,73634,73634,73634,73634,73634
FA,3,272668,272668,272662,272668,272668,272668,272668,272668,272668,272668,...,272668,272668,272668,272668,272668,272668,272668,272668,272668,272668
FC,4,41434,41434,41434,41434,41434,41434,41434,41434,41434,41434,...,41434,41434,41434,41434,41434,41434,41434,41434,41434,41434
FS,5,9722,9722,9722,9722,9722,9722,9722,9722,9722,9722,...,9722,9722,9722,9722,9722,9722,9722,9722,9722,9722
IB,6,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
KN,7,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
PO,8,54,54,54,54,54,54,54,54,54,54,...,54,54,54,54,54,54,54,54,54,54


In [156]:
data_ready = pitches.groupby(['GAMEID', 'TOP_BOTTOM']).agg({'values': lambda x: list(x), 'Next Pitch Label': lambda x: list(x)})
data_ready.head()

values  \
GAMEID TOP_BOTTOM                                                      
529466 Bottom      [[1.0, 1.0, 505.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0....   
       Top         [[1.0, 1.0, 525.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0....   
529615 Bottom      [[1.0, 1.0, 414.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0....   
       Top         [[1.0, 1.0, 61.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0...   
529630 Bottom      [[1.0, 1.0, 258.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0....   

                                                    Next Pitch Label  
GAMEID TOP_BOTTOM                                                     
529466 Bottom      [3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 2.0, 3.0, 2.0, ...  
       Top         [3.0, 3.0, 3.0, 2.0, 10.0, 10.0, 10.0, 0.0, 10...  
529615 Bottom      [3.0, 3.0, 2.0, 0.0, 3.0, 0.0, 3.0, 0.0, 2.0, ...  
       Top         [0.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 10.0,...  
529630 Bottom      [10.0, 10.0, 2.0, 10.0, 0.0, 10.0, 3.0, 10.0, ...

In [157]:
train, test = train_test_split(data_ready, test_size=0.2)

In [158]:
train_values = train['values'].values
train_labels = train['Next Pitch Label'].values
test_values = test['values'].values
test_labels = test['Next Pitch Label'].values

In [159]:
max_length = 300

In [160]:
# add code to prevent games with > 200 pitches from being considered 
def generate_batch(batch_size, values, labels):
    
    total = len(values)
    
    indices = np.random.choice(total, batch_size, replace=False)
    
    batch_values = np.take(values, indices)
    
    batch_labels = np.take(labels, indices)
    
    to_be_masked = [0 for _ in range(input_size)]
    
    masked = []
    
    values = []
    for val in batch_values:
        m = [1 for _ in range(len(val))]
        while len(val) < max_length:
            val.append(to_be_masked)
            m.append(0)
        values.append(val)
        masked.append(m)
    
    
    labels = []
    for val in batch_labels:
        while len(val) < max_length:
            val.append(0)
        labels.append(val)
    
    return values, labels, masked

In [161]:
v, l, masked = generate_batch(20, train_values, train_labels)

In [162]:
input_size = len(input_cols) - 1
output_size = 14

In [163]:
class LSTMModel():
    
    def __init__(self, rnn_size, output_size, learning_rate=1e-4):

        self.inputs = tf.placeholder(tf.float32, shape=[None, max_length, input_size]) # batch size, num_pitches, num_inputs
        self.labels = tf.placeholder(tf.int32, shape=[None, max_length])
        self.labels_mask = tf.placeholder(tf.bool, shape=[None, max_length])
    
        lm_cell = tf.nn.rnn_cell.LSTMCell(rnn_size)
    
        outputs, states = tf.nn.dynamic_rnn(lm_cell, self.inputs, dtype=tf.float32)
    
        self.output_logits = tf.layers.dense(outputs, output_size)
    
        self.loss = tf.losses.sparse_softmax_cross_entropy(self.labels, self.output_logits, weights=tf.cast(self.labels_mask, tf.float32))
        
        optimizer = tf.train.AdamOptimizer(learning_rate)
        
        self.global_step = tf.train.get_or_create_global_step()
        self.train_op = optimizer.minimize(self.loss)
        self.saver = tf.train.Saver()

In [164]:
tf.reset_default_graph()
model = LSTMModel(256, output_size, 1e-3)

In [166]:
inputs, labels, masked = generate_batch(20, train_values, train_labels)
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    d = {model.inputs: inputs, model.labels: labels, model.labels_mask: masked}
    
    for i in range(10):
    
        loss, _ = sess.run([model.loss, model.train_op], feed_dict = d)
    
        print(loss)

2.9172835
2.60938
2.369035
2.2195237
2.1006098
2.0072916
1.9325368
1.8691199
1.8153536
1.7714014
